# Setup

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage.transform import resize

#convert to labels
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense,InputLayer
import os

from keras.models import load_model

from keras.callbacks import EarlyStopping


from keras.applications.inception_resnet_v2 import preprocess_input


from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.layers import BatchNormalization,Dropout

from keras.preprocessing import image
import os



In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Loading Data

## Defining Directory

In [0]:
path = "/content/drive/My Drive/Analytics Vidhya Course/Computer Vision/CV github repo/datasets/male_vs_female_classification" 
os.chdir(path)
img_dir = path + "/images/"

In [0]:
#path of files

csv_file = path + "/train.csv"

print(csv_file)
print(img_dir)

/content/drive/My Drive/Analytics Vidhya Course/Computer Vision/CV github repo/datasets/male_vs_female_classification/train.csv
/content/drive/My Drive/Analytics Vidhya Course/Computer Vision/CV github repo/datasets/male_vs_female_classification/images/


## Loading csv file

In [0]:
data = pd.read_csv(csv_file)

In [0]:
data['class'].value_counts(normalize=True) * 100

1    50.623155
0    49.376845
Name: class, dtype: float64

# Dividing data into training and test set

In [0]:
from sklearn.utils import shuffle
#split data into ones and zeroes

data_ones = data[ data['class'] == 1]
data_zeroes = data[ data['class'] == 0]

# randomly shuffle both splits
data_ones = shuffle(data_ones)
data_zeroes = shuffle(data_zeroes)


# combine first 70% into train and last 30% into test.
train_set = shuffle(data_ones[:int(0.7*len(data_ones))].append(data_zeroes[:int(0.7*len(data_zeroes))]))
test_set =  shuffle(data_ones[int(0.7*len(data_ones)):].append(data_zeroes[int(0.7*len(data_zeroes)):]))

print("for training set")
print(train_set['class'].value_counts(normalize=True) * 100)
print(train_set.head())

print("for test set")
print(test_set['class'].value_counts(normalize=True) * 100)
print(test_set.head())


for training set
1    50.6209
0    49.3791
Name: class, dtype: float64
      image_names  class
11901    2139.jpg      1
5883     1528.jpg      1
5759     6085.jpg      1
179      5604.jpg      1
466      9477.jpg      1
for test set
1    50.628415
0    49.371585
Name: class, dtype: float64
     image_names  class
1161   14785.jpg      0
3530    2315.jpg      1
3007   15594.jpg      0
4330     854.jpg      0
6034    3454.jpg      1


In [0]:
def load_data(data):
  # load images and store it in numpy array
  X = []

  for img_name in data.image_names:
      img = plt.imread(img_dir + img_name)
      X.append(img)

  # store outputs
  y = data[['class']].values
  return np.asarray(X),np.asarray(to_categorical(y))

In [0]:
X_valid,y_valid = load_data(test_set)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650


In [0]:
X_train,y_train = load_data(train_set)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900


# Pre-Processing data.


In [0]:
def pre_process(X):
  return preprocess_input(X) #preprocess_input(X,mode='tf')

In [0]:
X_valid = pre_process(X_valid)

In [0]:
X_train = pre_process(X_train)

# Loading pre-trained models

In [0]:
#base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
base_model = InceptionResNetV2(weights='imagenet', input_shape=(224, 224, 3), include_top=False) 














219062272/219055592 [==============================] - 3s 0us/step













219062272/219055592 [==============================] - 3s 0us/step


# Fine tune model for current problem
Steps:
1. Extract features
2. Reshape the data
3. Rescale features
4. Create top model
5. Train the model

In [0]:
# Extract features using the pre-trained  model

# for training set
base_model_pred = base_model.predict(X_train)

#for validation set
base_model_pred_valid = base_model.predict(X_valid)

In [0]:
train_shape = base_model_pred.shape

In [0]:
base_model_pred = base_model_pred.reshape(train_shape[0], train_shape[1]*train_shape[2]*train_shape[3])

In [0]:
valid_shape = base_model_pred_valid.shape

In [0]:
print(train_shape)
print(valid_shape)

(8536, 5, 5, 1536)
(3660, 5, 5, 1536)
(8536, 5, 5, 1536)
(3660, 5, 5, 1536)


In [0]:
base_model_pred_valid = base_model_pred_valid.reshape(valid_shape[0], valid_shape[1]*valid_shape[2]*valid_shape[3])

In [0]:
# checking min and max for extracted features.
base_model_pred.min(),base_model_pred.max()

(0.0, 33.474297)

(0.0, 33.474297)

In [0]:
max_val = base_model_pred.max()

Let us ensure that the high variation in pixel values present(if any), does not cause any issues.

In [0]:
# Normalizing  features to [0,1]
base_model_pred = base_model_pred/max_val
base_model_pred.min(),base_model_pred.max()

(0.0, 1.0)

(0.0, 1.0)

### Defining final layers of architechture

In [0]:
model = Sequential()
model.add(InputLayer((train_shape[1]*train_shape[2]*train_shape[3], )))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(units=2, activation='softmax'))

In [0]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_5 (Dropout)          (None, 38400)             0         
_________________________________________________________________
batch_normalization_208 (Bat (None, 38400)             153600    
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              39322624  
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_209 (Bat (None, 1024)              4096      
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 2050      
Total params: 39,482,370
Trainable params: 39,403,522
Non-trainable params: 78,848
_____________________________________

In [0]:
estop = EarlyStopping(monitor='val_loss', mode='min',patience=5)

In [0]:
callbacks_list = [estop]

In [0]:
history = model.fit(base_model_pred, y_train, epochs=50, validation_data=(base_model_pred_valid, y_valid),callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 8536 samples, validate on 3660 samples
Epoch 1/50
1952/8536 [=====>........................] - ETA: 50s - loss: 0.7057 - acc: 0.7556WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 8536 samples, validate on 3660 samples
Epoch 1/50
8536/8536 [==============================] - 56s 7ms/step - loss: 0.5023 - acc: 0.7970 - val_loss: 2.2153 - val_acc: 0.8470
Epoch 2/50
8536/8536 [==============================] - 50s 6ms/step - loss: 0.2886 - acc: 0.8769 - val_loss: 2.4647 - val_acc: 0.8342
Epoch 3/50
8536/8536 [==============================] - 50s 6ms/step - loss: 0.2127 - acc: 0.9100 - val_loss: 2.2221 - val_acc: 0.8486
Epoch

In [0]:
model.save_weights("weights.h5")

Plotting accuracy

In [0]:
plt.plot(history.history['acc'], 'r')
plt.plot(history.history['val_acc'], 'b')
plt.legend({'training accuracy': 'r', 'validation accuracy':'b'})
plt.show()


Plotting loss

In [0]:
plt.plot(history.history['loss'], 'r')
plt.plot(history.history['val_loss'], 'b')
plt.legend({'training loss': 'r', 'validation loss':'b'})
plt.show()

# Make predictions on new data

In [0]:
csv_file = path + "/test.csv"

In [0]:
data = pd.read_csv(csv_file)

In [0]:
def load_data(data):
  # load images and store it in numpy array
  X = []
  for img_name in data.image_names:
      filename = img_dir + img_name
      #img = plt.imread(img_dir + img_name)
      img = image.load_img(filename, target_size=(224, 224))
      img =image.img_to_array(img)
      X.append(img)

  return np.asarray(X)

In [0]:
X = load_data(data)

## Pre-Processing Data and loading pre-trained model

In [0]:
preprocess_input(X)

array([[[[ 0.13725495, -0.9529412 , -0.7882353 ],
         [ 0.13725495, -0.9529412 , -0.7882353 ],
         [ 0.13725495, -0.9529412 , -0.7882353 ],
         ...,
         [-0.38039213, -0.45098037, -0.31764704],
         [-0.5764706 , -0.64705884, -0.5137255 ],
         [-0.58431375, -0.654902  , -0.52156866]],

        [[ 0.13725495, -0.9529412 , -0.7882353 ],
         [ 0.13725495, -0.9529412 , -0.7882353 ],
         [ 0.13725495, -0.9529412 , -0.7882353 ],
         ...,
         [-0.41176468, -0.4823529 , -0.3490196 ],
         [-0.5921569 , -0.6627451 , -0.5294118 ],
         [-0.5921569 , -0.6627451 , -0.5294118 ]],

        [[ 0.13725495, -0.9529412 , -0.7882353 ],
         [ 0.13725495, -0.9529412 , -0.7882353 ],
         [ 0.13725495, -0.9529412 , -0.7882353 ],
         ...,
         [-0.4352941 , -0.5058824 , -0.372549  ],
         [-0.58431375, -0.654902  , -0.52156866],
         [-0.5764706 , -0.64705884, -0.5137255 ]],

        ...,

        [[ 0.05882359, -0.31764704, -0

In [0]:
base_model = InceptionResNetV2(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

In [0]:
base_model_pred = base_model.predict(X)
data_shape = base_model_pred.shape
base_model_pred = base_model_pred.reshape(data_shape[0], data_shape[1]*data_shape[2]*data_shape[3])
max_val = base_model_pred.max()
base_model_pred = base_model_pred/max_val
print(base_model_pred.min(),base_model_pred.max())

0.0 1.0


In [0]:
print(data_shape)

(5227, 5, 5, 1536)


In [0]:
print(train_shape)

(8536, 5, 5, 1536)


In [0]:
#data_shape = train_shape
model = Sequential()
model.add(InputLayer((data_shape[1]*data_shape[2]*data_shape[3], )))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(units=2, activation='softmax'))

In [0]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [0]:
model.load_weights("weights.h5")

## Make Predictions

In [0]:
y = model.predict_classes(base_model_pred)

In [0]:
print(type(y))

<class 'numpy.ndarray'>


In [0]:
output = list(y)
#print(data.head())

In [0]:
data['output'] = output

In [0]:
print(data.head())

  image_names  output
0    5603.jpg       1
1    1286.jpg       1
2    9815.jpg       1
3    8865.jpg       0
4    9306.jpg       1


### Storing output

In [0]:
data.to_csv("output.csv",index = False)